<a href="https://colab.research.google.com/github/nsajwan/line_item_extraction/blob/master/Line_Item_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bank statements Line Item Extraction
Based on Cascade TableNet - https://github.com/DevashishPrasad/CascadeTabNet 

## 1. Install all the prerequisite
This will take a while, Remember to **Restart Runtime** after this step is done 

In [2]:
!pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q mmcv terminaltables
!git clone --branch v1.2.0 'https://github.com/open-mmlab/mmdetection.git'
%cd "mmdetection"
!pip install -r "/content/mmdetection/requirements/optional.txt"
!python setup.py install
!python setup.py develop
!pip install -r {"requirements.txt"}
!pip install pillow==6.2.1 
!pip install mmcv==0.4.3
%cd "/content"
!apt-get install poppler-utils
!pip install pdf2image
!pip install pdfminer.six
!pip install Pillow
!pip install pdfrw

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9MB 25kB/s 
     |████████████████████████████████| 4.1MB 50.4MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101
     |████████████████████████████████| 245kB 13.3MB/s 
     |████████████████████████████████| 194kB 51.4MB/s 
Cloning into 'mmdetection'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13137 (delta 1), reused 1 (delta 0), pack-reused 13128
Receiving objects: 100% (13137/13137), 13.71 MiB | 5.98 MiB/s, done.
Resolving deltas: 100% (8873/8873), done.
Note: checking out '0f33c08d8d46eba8165715a0995841a975badfd4'.

You are in 'detach

Streaming output truncated to the last 5000 lines.
   ^~~~~~~~
mmdet/ops/masked_conv/src/masked_conv2d_cuda.cpp:25:3: note: in expansion of macro ‘CHECK_CONTIGUOUS’
   CHECK_CONTIGUOUS(x)
   ^~~~~~~~~~~~~~~~
mmdet/ops/masked_conv/src/masked_conv2d_cuda.cpp:34:3: note: in expansion of macro ‘CHECK_INPUT’
   CHECK_INPUT(col);
   ^
In file included from /usr/local/lib/python3.6/dist-packages/torch/include/c10/core/Device.h:5:0,
                 from /usr/local/lib/python3.6/dist-packages/torch/include/c10/core/Allocator.h:6,
                 from /usr/local/lib/python3.6/dist-packages/torch/include/ATen/ATen.h:3,
                 from /usr/local/lib/python3.6/dist-packages/torch/include/torch/csrc/api/include/torch/types.h:3,
                 from /usr/local/lib/python3.6/dist-packages/torch/include/torch/csrc/api/include/torch/data/dataloader_options.h:4,
                 from /usr/local/lib/python3.6/dist-packages/torch/include/torch/csrc/api/include/torch/data/dataloader/base.h:3,
    

     |████████████████████████████████| 2.1MB 13.0MB/s 
  Found existing installation: Pillow 6.2.2
    Uninstalling Pillow-6.2.2:
      Successfully uninstalled Pillow-6.2.2


     |████████████████████████████████| 61kB 7.0MB/s 
  Created wheel for mmcv: filename=mmcv-0.4.3-cp36-cp36m-linux_x86_64.whl size=170847 sha256=84499988ab2d1e297ce716c859b16c922a3024d64a62bbb05a4b4cabfac60c8c
  Stored in directory: /root/.cache/pip/wheels/1e/c3/ee/7eddece4815cfe949be3d489912d22465e244260b587bb05e1
Successfully built mmcv
  Found existing installation: mmcv 1.1.2
    Uninstalling mmcv-1.1.2:
      Successfully uninstalled mmcv-1.1.2
/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils

## 2. Clone the Repository

In [80]:
!rm -rf line_item_extraction
!git clone https://github.com/strangest-quark/line_item_extraction.git

Cloning into 'line_item_extraction'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 93 (delta 41), reused 45 (delta 12), pack-reused 0
Unpacking objects: 100% (93/93), done.


## 3. Pdf to Images and XML


In [107]:
!rm -rf /content/line_item_extraction/results
import os
from pdf2image import convert_from_path, convert_from_bytes
from PIL import Image
import subprocess

from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

pdf_directory = '/content/line_item_extraction/sample_docs'
res_directory = '/content/line_item_extraction/results'
#cmd2 = 'pdf2txt.py -t xml '
cmd2 = 'pdf2txt.py -t xml --line-marg 1.0 --char-margin 1.0 '
results = []

for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
      os.makedirs(res_directory+'/'+filename[:-4])
      full_cmd = cmd2+ '-o ' + res_directory+'/'+filename[:-4] + '/' +'poppler.xml ' + pdf_directory+'/'+filename
      # Generate poppler xml
      !{full_cmd}
      images = convert_from_path(pdf_directory+'/'+filename)
      i = 0
      # create result dirs
      os.makedirs(res_directory+'/'+filename[:-4]+'/input_img')
      os.makedirs(res_directory+'/'+filename[:-4]+'/output_img')
      os.makedirs(res_directory+'/'+filename[:-4]+'/output_xml')
      for image in images:
        # save images
        image.save(res_directory+'/'+filename[:-4]+'/input_img/'+str(i)+'.png', format='PNG')
        i=i+1



## 3. Download the Pretrained Model


In [66]:
!gdown "https://drive.google.com/u/0/uc?id=1-QieHkR1Q7CXuBu4fp3rYrvDG9j26eFT"

Downloading...
From: https://drive.google.com/u/0/uc?id=1-QieHkR1Q7CXuBu4fp3rYrvDG9j26eFT
To: /content/epoch_36.pth
664MB [00:09, 73.2MB/s]


## 4. Run the Predictions

In [108]:
from mmdet.apis import init_detector, inference_detector, show_result
import matplotlib.pyplot as plt 
import mmcv

def show_result_pyplot(img, result, class_names, out_file, score_thr=0.3, fig_size=(15, 10)):
    print(img,"---" , class_names, "---" ,out_file,"---" , score_thr,"---" , fig_size)
    img = show_result(img, result, class_names, score_thr=score_thr, show=False, out_file=out_file)

# Load model
config_file = '/content/line_item_extraction/config/cascade_mask_rcnn_hrnetv2p_w32_20e.py'
checkpoint_file = '/content/epoch_36.pth'
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

directory = '/content/line_item_extraction/results/'

results = dict()

for folder in os.listdir(directory):
  result = dict()
  for filename in os.listdir(directory+'/'+folder+'/input_img'):
      if filename.endswith(".png"):
        # Run Inference
        res = inference_detector(model, directory+'/'+folder+'/input_img/'+filename)
        result[filename[:-4]] = res
        # Visualize results - stored in /results/filename/output_img/
        try:
          show_result_pyplot(directory+folder+'/input_img/'+filename, res,('Bordered', 'cell', 'Borderless'), out_file=directory+folder+'/output_img/'+filename, score_thr=0.85)
        except:
          print(filename, folder, " Error - probably empty image")
  results[folder] = result
  print("Success! Segmentation results stored in results/filename/output_img")

/content/line_item_extraction/results/Redacted_CIBC_2_Sample2/input_img/0.png --- ('Bordered', 'cell', 'Borderless') --- /content/line_item_extraction/results/Redacted_CIBC_2_Sample2/output_img/0.png --- 0.85 --- (15, 10)
/content/line_item_extraction/results/Redacted_CIBC_2_Sample2/input_img/1.png --- ('Bordered', 'cell', 'Borderless') --- /content/line_item_extraction/results/Redacted_CIBC_2_Sample2/output_img/1.png --- 0.85 --- (15, 10)
1.png Redacted_CIBC_2_Sample2  Error - probably empty image
Success! Segmentation results stored in results/filename/output_img
/content/line_item_extraction/results/RedactedPayPal_sample3/input_img/0.png --- ('Bordered', 'cell', 'Borderless') --- /content/line_item_extraction/results/RedactedPayPal_sample3/output_img/0.png --- 0.85 --- (15, 10)
/content/line_item_extraction/results/RedactedPayPal_sample3/input_img/2.png --- ('Bordered', 'cell', 'Borderless') --- /content/line_item_extraction/results/RedactedPayPal_sample3/output_img/2.png --- 0.85 -

After this, use TSR postprocessing for best result

In [109]:
%cd "/content"
from line_item_extraction.border_main import border
import lxml.etree as etree
import cv2
from google.colab.patches import cv2_imshow
from mmdet.apis import inference_detector, show_result, init_detector
import glob
from line_item_extraction.cell_text import borderless

config_fname = "/content/line_item_extraction/config/cascade_mask_rcnn_hrnetv2p_w32_20e.py" 
checkpoint_path = "/content/"
epoch = 'epoch_36.pth'

model = init_detector(config_fname, checkpoint_path+epoch)

for folder in os.listdir(directory):
  for filename in os.listdir(directory+folder+'/input_img'):
      i = directory+folder+'/input_img/'+filename
      print("Full path of image i=",i)
      result = results[folder][filename[:-4]]
      try:
        result = inference_detector(model, i)
        res_border = []
        res_bless = []
        res_cell = []
        root = etree.Element("document")
        ## for border
        for r in result[0][0]:
          if r[4] > .85:
              res_border.append(r[:4].astype(int))
        ## for cells
        for r in result[0][1]:
            if r[4] > .85:
                r[4] = r[4] * 100
                res_cell.append(r.astype(int))
        ## for borderless
        for r in result[0][2]:
            if r[4] > .85:
                res_bless.append(r[:4].astype(int))

        ## if border tables detected
        if len(res_border) != 0:
            ## call border script for each table in image
            for res in res_border:
                try:
                    root.append(border(res, cv2.imread(i)))
                except:
                    pass
        if len(res_bless) != 0:
            if len(res_cell) != 0:
                for no, res in enumerate(res_bless):
                    root.append(borderless(res, cv2.imread(i), res_cell))
        print("myfile=",directory+folder+'/output_xml/'+ i.split('/')[-1][:-3] + 'xml')            
        myfile = open(directory+folder+'/output_xml/'+ i.split('/')[-1][:-3] + 'xml', "w")
        myfile.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        myfile.write(etree.tostring(root, pretty_print=True, encoding="unicode"))
        myfile.close()
      except:
        print("Error")

/content
Full path of image i= /content/line_item_extraction/results/Redacted_CIBC_2_Sample2/input_img/0.png
[Table status] : Processing table with lines
myfile= /content/line_item_extraction/results/Redacted_CIBC_2_Sample2/output_xml/0.xml
Full path of image i= /content/line_item_extraction/results/Redacted_CIBC_2_Sample2/input_img/1.png
myfile= /content/line_item_extraction/results/Redacted_CIBC_2_Sample2/output_xml/1.xml
Full path of image i= /content/line_item_extraction/results/RedactedPayPal_sample3/input_img/0.png
[Table status] : Processing table with lines
[Table status] : Processing table with lines
[Table status] : Processing table with lines
myfile= /content/line_item_extraction/results/RedactedPayPal_sample3/output_xml/0.xml
Full path of image i= /content/line_item_extraction/results/RedactedPayPal_sample3/input_img/2.png
[Table status] : Processing table with lines
myfile= /content/line_item_extraction/results/RedactedPayPal_sample3/output_xml/2.xml
Full path of image i= 

In [113]:
import lxml.etree as et
import numpy


# def combine_xmls(files):
#     xml_files = glob.glob(files +"/*.xml")
#     xml_element_tree = None
#     for xml_file in xml_files:
#         data = et.parse(xml_file).getroot()
#         if xml_element_tree is None:
#             xml_element_tree = data 
#         else:
#             xml_element_tree.extend(data) 
#     if xml_element_tree is not None:
#         return xml_element_tree

# def combine_page_xmls2():
#   for folder in os.listdir(directory):
#     document = etree.Element("document")
#     xml_out = combine_xmls(directory+folder+'/output_xml')
#     myfile = open(directory+folder+'/combinedOutput.xml', "w")
#     myfile.write('<?xml version="1.0" encoding="UTF-8"?>\n')
#     myfile.write(etree.tostring(xml_out, pretty_print=True, encoding="unicode"))
#     myfile.close()
#     print(etree.tostring(xml_out, pretty_print=True).decode())

def combine_page_xmls():
  for folder in os.listdir(directory):
    document = etree.Element("document")
    for filename in sorted(os.listdir(directory+folder+'/output_xml')):
      print("filename--",filename)
      tree = et.ElementTree(file=directory+folder+'/output_xml/'+filename)
      pageNode = etree.SubElement(document, 'page')
      tables = tree.getroot().getchildren()
      for table in tables:
        pageNode.append(table)
      
    myfile = open(directory + folder + '/combinedOutput.xml', "w")
    myfile.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    myfile.write(etree.tostring(document, pretty_print=True, encoding="unicode"))
    myfile.close()
    print(etree.tostring(document, pretty_print=True).decode())

combine_page_xmls()


def TranslateFromMachineCode(src, pdf, im, pageNo):
        sx0, sy0, sx1, sy1 = src
        pdfWidth, pdfHeight = (int(float(pdf.pages[pageNo].MediaBox[2])),int(float(pdf.pages[pageNo].MediaBox[3])))
        imageWidth, imageHeight = im.size # size of png image 612x1008.  #1700x2800
        sy01 = imageHeight-sy1
        sy11 = imageHeight-sy0
        x0 = sx0/imageWidth*pdfWidth
        x1 = sx1/imageWidth*pdfWidth
        y0 = sy01/imageHeight*pdfHeight
        y1 = sy11/imageHeight*pdfHeight
        return (x0, y0, x1, y1)

def getText(textline):
  txt = ''
  for text in textline.getchildren():
      txt = txt+text.text
  return txt

def getTextFromPdf(points, pdf, page, folder):
  found = False
  tree = et.ElementTree(file=directory+folder+'/poppler.xml')
  page = tree.getroot().getchildren()[page]
  textboxes = page.getchildren()
  for textbox in textboxes:
    textlines = textbox.getchildren()
    for textline in textlines:
      if 'bbox' not in textline.attrib:
        continue
      coods = textline.attrib['bbox'].split(',')
      coods = [float(i) for i in coods] 
      points = [float(i) for i in points]
      diff = numpy.sum((numpy.asarray(points)-numpy.asarray(coods))**2)
      if diff<30:
        found = True
        txt = getText(textline)
        #print(coods, points, numpy.sum((numpy.asarray(points)-numpy.asarray(coods))**2), txt)
        return txt
  if not found:
    return found


filename-- 0.xml
filename-- 1.xml
<document>
  <page><table>
    <Coords points="150,1318 150,1486 1622,1486 1622,1318"/>
    <cell end-col="1" end-row="1" start-col="1" start-row="1">
      <Coords points="291,1348 291,1371 430,1371 430,1348"/>
    </cell>
    <cell end-col="2" end-row="1" start-col="2" start-row="1">
      <Coords points="817,1348 817,1371 1004,1371 1004,1348"/>
    </cell>
    <cell end-col="3" end-row="1" start-col="3" start-row="1">
      <Coords points="1120,1349 1120,1373 1265,1373 1265,1349"/>
    </cell>
    <cell end-col="4" end-row="1" start-col="4" start-row="1">
      <Coords points="1433,1348 1433,1372 1567,1372 1567,1348"/>
    </cell>
    <cell end-col="1" end-row="2" start-col="1" start-row="2">
      <Coords points="291,1385 291,1407 494,1407 494,1385"/>
    </cell>
    <cell end-col="4" end-row="2" start-col="4" start-row="2">
      <Coords points="1488,1384 1488,1404 1581,1404 1581,1384"/>
    </cell>
    <cell end-col="2" end-row="3" start-col="2" 

In [114]:
import xml.etree.ElementTree as et
import lxml.etree as etree
from pdfrw import PdfReader
from PIL import Image

for folder in os.listdir(directory):
  tree = et.ElementTree(file=directory+folder+'/combinedOutput.xml')
  #tree = et.ElementTree(file='/content/line_item_extraction/results/0.xml')
  pages = tree.getroot().getchildren()
  foundCount = 0
  notFoundCount = 0
  bank_statement = etree.Element("bank_statement")
  for page in pages:
    for table in page:
      cells = table.getchildren()[1:]
      maxCol = 1;
      for cell in cells:
        if int(cell.attrib['end-col']) > maxCol:
          maxCol = int(cell.attrib['end-col'])
      if maxCol > 2:   
        table.set("lineItems", "true")
      else:
        table.set("lineItems", "false")  

  pageNo = 0
  for page in pages:
    for table in page:
      print(folder,"pageNo",pageNo)
      if table.attrib['lineItems'] == "true":
        line_items = etree.SubElement(bank_statement, 'line_items')
      else:  
        line_items = etree.SubElement(bank_statement, 'others')
      cells = table.getchildren()[1:]
      key = ''
      startRow = 999
      headers = etree.SubElement(line_items, 'headers')
      headerStr = ""
      for cell in cells:
        coords = cell.getchildren()
        points = coords[0].attrib['points']
        point0 = int(points.split(' ')[0].split(',')[0])
        point1 = int(points.split(' ')[0].split(',')[1])
        point2 = int(points.split(' ')[2].split(',')[0])
        point3 = int(points.split(' ')[2].split(',')[1])


        pdf = PdfReader(pdf_directory+'/' + folder + '.pdf')
        im = Image.open(directory + folder+'/output_img/'+str(pageNo)+'.png')

        text = getTextFromPdf(TranslateFromMachineCode((point0,point1,point2,point3), pdf, im, pageNo), pdf, pageNo, folder)

        if table.attrib['lineItems'] == "true":
          if cell.attrib['start-row'] == '0':
            if text == False:
              text = 'unknown'
            headerStr = "".join((headerStr, ', '+str(text)))
          else:
            if startRow != int(cell.attrib['start-row']):
              row = etree.SubElement(line_items, 'row')
            startRow = int(cell.attrib['start-row'])
            if text == False:
              keys = etree.SubElement(row, 'key')
              keys.text='unknown'
            else:  
              keys = etree.SubElement(row, 'key')
              keys.text=text.strip()
        else:
          if cell.attrib['start-col'] == '0':
            if text == False:
              key = etree.SubElement(line_items, 'unknown')
            else:  
              tmpKey = ''.join(filter(str.isalpha, str(text)))
              if tmpKey == '':
                key = etree.SubElement(line_items, 'unknown')
              else:  
                key = etree.SubElement(line_items, tmpKey)
          else:
            if text == False:
              key.text='unknown'
            else:  
              key.text=text

        if text != False:
          foundCount = foundCount+1
          textNode = et.Element("text")
          textNode.text = text
          cell.append(textNode)
        else:
          notFoundCount = notFoundCount+1
      if len(headerStr) > 0:
        headers.text =  headerStr
    pageNo = pageNo +1
  print('mapped='+str(foundCount)+' no matches='+str(notFoundCount))
  myfile = open(directory+folder+'/bank_statement.xml', "w")
  myfile.write('<?xml version="1.0" encoding="UTF-8"?>\n')
  myfile.write(etree.tostring(bank_statement, pretty_print=True, encoding="unicode"))
  myfile.close()
  print(etree.tostring(bank_statement, pretty_print=True).decode())





Redacted_CIBC_2_Sample2 pageNo 0
mapped=10 no matches=0
<bank_statement>
  <line_items>
    <headers/>
    <row>
      <key>Description</key>
      <key>Withdrawals ($)</key>
      <key>Deposits ($)</key>
      <key>Balance ($)</key>
    </row>
    <row>
      <key>Opening balance</key>
      <key>$126.29</key>
    </row>
    <row>
      <key>0.01</key>
      <key>126.28</key>
    </row>
    <row>
      <key>Closing balance</key>
      <key>$126.28</key>
    </row>
  </line_items>
</bank_statement>

RedactedPayPal_sample3 pageNo 0
RedactedPayPal_sample3 pageNo 0
RedactedPayPal_sample3 pageNo 0
RedactedPayPal_sample3 pageNo 1
RedactedPayPal_sample3 pageNo 2
mapped=114 no matches=14
<bank_statement>
  <line_items>
    <headers>, Tran Date
, unknown, Reference Number
, Description
,  Amount
</headers>
    <row>
      <key>05/15/20</key>
      <key>05/15/20</key>
      <key>P928300GS01DXNVTM</key>
      <key>unknown</key>
      <key>-$276.59</key>
      <key>unknown</key>
    </row>
    <r